# Prepare batch scoring model

In this notebook, you will prepare a model used to detect suspicious activity that will be used for batch scoring.

The team at Woodgrove Bank has provided you with exported CSV copies of historical data for you to train your model against. Run the following cell to load required libraries and download the data sets from the Azure ML datastore.

In [ ]:
#!pip install --upgrade azureml-train-automl-runtime==1.36.0
#!pip install --upgrade azureml-automl-runtime==1.36.0
#!pip install --upgrade scikit-learn
#!pip install --upgrade numpy

In [ ]:
from azureml.core import Workspace, Environment, Datastore, Dataset
from azureml.core.experiment import Experiment
from azureml.core.run import Run
from azureml.core.model import Model

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# sklearn.externals.joblib was deprecated in 0.21
from sklearn import __version__ as sklearnver
from packaging.version import Version
if Version(sklearnver) < Version("0.21.0"):
    from sklearn.externals import joblib
else:
    import joblib

import numpy as np
import pandas as pd

ws = Workspace.from_config()

# Load data
ds = Datastore.get(ws, "woodgrovestorage")

account_ds = Dataset.Tabular.from_delimited_files(path = [(ds, 'synapse/Account_Info.csv')])
fraud_ds = Dataset.Tabular.from_delimited_files(path = [(ds, 'synapse/Fraud_Transactions.csv')])
untagged_ds = Dataset.Tabular.from_delimited_files(path = [(ds, 'synapse/Untagged_Transactions.csv')])

# Create pandas dataframes from datasets
account_df = account_ds.to_pandas_dataframe()
fraud_df = fraud_ds.to_pandas_dataframe()
untagged_df = untagged_ds.to_pandas_dataframe()

In [ ]:
print(sklearnver)

In [ ]:
from azureml.core import __version__ as amlver
print(amlver)

In [ ]:
pip freeze

In [ ]:
###### Reorder the column of dataframe by ascending order in pandas 
cols=untagged_df.columns.tolist()
cols.sort()
untagged_df=untagged_df[cols]

## Prepare accounts

Remove columns that have very few or no values: `accountOwnerName`, `accountAddress`, `accountCity` and `accountOpenDate` 

In [ ]:
account_df_clean = account_df[["accountID", "transactionDate", "transactionTime", 
                               "accountPostalCode", "accountState", "accountCountry", 
                               "accountAge", "isUserRegistered", "paymentInstrumentAgeInAccount", 
                               "numPaymentRejects1dPerUser"]]
account_df_clean = account_df_clean.copy()

Cleanup `paymentInstrumentAgeInAccount`. Values that are not numeric, are converted to NaN and then we can fill those NaN values with 0.

In [ ]:
account_df_clean['paymentInstrumentAgeInAccount'] = pd.to_numeric(account_df_clean['paymentInstrumentAgeInAccount'], errors='coerce')
account_df_clean['paymentInstrumentAgeInAccount'] = account_df_clean[['paymentInstrumentAgeInAccount']].fillna(0)['paymentInstrumentAgeInAccount']

Next, let's convert the `numPaymentRejects1dPerUser` so that the column has a datatype of `float` instead of `object`.

In [ ]:
account_df_clean["numPaymentRejects1dPerUser"] = account_df_clean[["numPaymentRejects1dPerUser"]].astype(float)["numPaymentRejects1dPerUser"]
account_df_clean["numPaymentRejects1dPerUser"].value_counts()

You need to combine the `transactionDate` and `transactionTime` fields into a single field `transactionDateTime`. Begin by converting the transactionTime from an integer to a 0 padded six digit string of the format hhmmss (2 digit hour minute second), then concatenate the two columns and finally parse the concatenated string as a DateTime value.

In [ ]:
account_df_clean["transactionTime"] = ['{0:06d}'.format(x) for x in account_df_clean["transactionTime"]]
account_df_clean["transactionDateTime"] = pd.to_datetime(account_df_clean["transactionDate"].map(str) + account_df_clean["transactionTime"], format='%Y%m%d%H%M%S')
account_df_clean["transactionDateTime"]

`account_df_clean` is now ready for use in modeling.

## Prepare untagged transactions

There are 16 columns in the untagged_transactions whose values are all null. Drop these columns to simplify the dataset. 

In [ ]:
untagged_df_clean = untagged_df.dropna(axis=1, how="all").copy()

Replace null values in `localHour` with `-99`. Also replace values of `-1` with `-99`.

In [ ]:
untagged_df_clean["localHour"] = untagged_df_clean["localHour"].fillna(-99)
untagged_df_clean.loc[untagged_df_clean.loc[:,"localHour"] == -1, "localHour"] = -99
untagged_df_clean["localHour"].value_counts()

Clean up the remaining null fields:
- Fix missing values for location fields by setting them to `NA` for unknown. 
- Set `isProxyIP` to False
- Set `cardType` to `U` for unknown (which is a new level)
- Set `cvvVerifyResult` to `N` which means for those where the transaction failed because the wrong CVV2 number was entered ro no CVV2 numebr was entered, treat those as if there was no CVV2 match.

In [ ]:
untagged_df_clean = untagged_df_clean.fillna(value={"ipState": "NA", "ipPostcode": "NA", "ipCountryCode": "NA", 
                               "isProxyIP":False, "cardType": "U", 
                               "paymentBillingPostalCode" : "NA", "paymentBillingState":"NA",
                               "paymentBillingCountryCode" : "NA", "cvvVerifyResult": "N"
                              })

The `transactionScenario` column provides no insights because all rows have the same `A` value. Drop that column. Same idea for the `transactionType` column.

In [ ]:
del untagged_df_clean["transactionScenario"]
del untagged_df_clean["transactionType"]

Create the `transactionDateTime` in the same way as shown previously.

In [ ]:
untagged_df_clean["transactionTime"] = ['{0:06d}'.format(x) for x in untagged_df_clean["transactionTime"]]
untagged_df_clean["transactionDateTime"] = pd.to_datetime(untagged_df_clean["transactionDate"].map(str) + untagged_df_clean["transactionTime"], format='%Y%m%d%H%M%S')
untagged_df_clean["transactionDateTime"]

`untagged_df_clean` is now ready for use in modeling.

## Prepare fraud transactions

The `transactionDeviceId` has no meaningful values, so drop it. Also, fill NA values of the `localHour` field with -99 as we did for the untagged transactions.

In [ ]:
fraud_df_clean = fraud_df.copy()
del fraud_df_clean['transactionDeviceId']
fraud_df_clean["localHour"] = fraud_df_clean["localHour"].fillna(-99)

Next, add the transactionDateTime column to the fraud data set using the same approach that was used for the untagged dataset.

In [ ]:
fraud_df_clean["transactionTime"] = ['{0:06d}'.format(x) for x in fraud_df_clean["transactionTime"]]
fraud_df_clean["transactionDateTime"] = pd.to_datetime(fraud_df_clean["transactionDate"].map(str) + fraud_df_clean["transactionTime"], format='%Y%m%d%H%M%S')
fraud_df_clean["transactionDateTime"]

Next, remove any duplicate rows from the fraud data set. We identify a unique transaction by the features `transactionID`, `accountID`, `transactionDateTime` and `transactionAmount`.

In [ ]:
fraud_df_clean = fraud_df_clean.drop_duplicates(subset=['transactionID', 'accountID', 'transactionDateTime', 'transactionAmount'], keep='first')

`fraud_df_clean` is now ready for use in modeling.

## Enrich the untagged data with account data

In this section, you will join the untagged dataset with the account dataset to enrich each untagged example.

In [ ]:
latestTrans_df = pd.merge(untagged_df_clean, account_df_clean, on='accountID', suffixes=('_unt','_act'))

In [ ]:
latestTrans_df

In [ ]:
latestTrans_df = latestTrans_df[latestTrans_df['transactionDateTime_act'] <= latestTrans_df['transactionDateTime_unt']]

Find the latest record timestamp.

In [ ]:
latestTrans_df = latestTrans_df.groupby(['accountID','transactionDateTime_unt']).agg({'transactionDateTime_act':'max'})

In [ ]:
latestTrans_df

Join the latest transactions with the untagged data frame and then the account data frame.

In [ ]:
joined_df = pd.merge(untagged_df_clean, latestTrans_df, how='outer', left_on=['accountID','transactionDateTime'], right_on=['accountID','transactionDateTime_unt'])
joined_df

In [ ]:
joined_df = pd.merge(joined_df, account_df_clean, left_on=['accountID','transactionDateTime_act'], right_on=['accountID','transactionDateTime'])
joined_df

Pick out only the columns needed for the model.

In [ ]:
untagged_join_acct_df = joined_df[['transactionID', 'accountID', 'transactionAmountUSD', 'transactionAmount','transactionCurrencyCode', 'localHour',
          'transactionIPaddress','ipState','ipPostcode','ipCountryCode', 'isProxyIP', 'browserLanguage','paymentInstrumentType',
           'cardType', 'paymentBillingPostalCode', 'paymentBillingState', 'paymentBillingCountryCode', 'cvvVerifyResult',
           'digitalItemCount', 'physicalItemCount', 'accountPostalCode', 'accountState', 'accountCountry', 'accountAge',
           'isUserRegistered', 'paymentInstrumentAgeInAccount', 'numPaymentRejects1dPerUser', 'transactionDateTime_act'
          ]]
untagged_join_acct_df

Rename the columns to clean the names up and remove the suffixes.

In [ ]:
untagged_join_acct_df = untagged_join_acct_df.rename(columns={
                                      'transactionDateTime_act':'transactionDateTime'
                                     })

## Labeling fraud examples

First, get the fraud time period for each account. Do this by grouping the fraud data by `accountID`. 

In [ ]:
fraud_t2 = fraud_df_clean.groupby(['accountID']).agg({'transactionDateTime':['min','max']})

Give these new columns some more friendly names.

In [ ]:
fraud_t2.columns = ["_".join(x) for x in fraud_t2.columns.ravel()]

In [ ]:
fraud_t2

Now left join the untagged dataset with the fraud dataset.

In [ ]:
untagged_joinedto_ranges = pd.merge(untagged_join_acct_df, fraud_t2, on='accountID', how='left')
untagged_joinedto_ranges

Now we use the joined data to apply a label according to the following rules:
* accountID from untagged not found in fraud dataset at all tagged as 0, meaning not fraudulent.
* accountID from untagged found in fraud dataset, but the transactionDateTime is outside of the time range from the fraud dataset tagged as 2.
* accountID from untagged found in fraud dataset and the transactionDateTime is within the time range from the fraud dataset tagged as 1, meaning fraudulent. 

In [ ]:
def label_fraud_range(row):
    if (str(row['transactionDateTime_min']) != "NaT") and (row['transactionDateTime'] >= row['transactionDateTime_min']) and (row['transactionDateTime'] <= row['transactionDateTime_max']):
        return 1
    elif (str(row['transactionDateTime_min']) != "NaT") and row['transactionDateTime'] < row['transactionDateTime_min']:
        return 2
    elif (str(row['transactionDateTime_max']) != "NaT") and row['transactionDateTime'] > row['transactionDateTime_max']:
        return 2
    else:
        return 0

In [ ]:
tagged_df_clean = untagged_joinedto_ranges
tagged_df_clean['label'] = untagged_joinedto_ranges.apply(lambda row: label_fraud_range(row), axis=1)
tagged_df_clean

This leaves us with 1,170 fraudulent examples, 198,326 non-fraudulent examples, and 504 examples that we will ignore as having occured prior to or after the fraud.

In [ ]:
tagged_df_clean['label'].value_counts()

Remove those examples with label value of 2 and drop the features `transactionDateTime_min` and `transactionDateTime_max`

In [ ]:
tagged_df_clean = tagged_df_clean[tagged_df_clean['label'] != 2]
del tagged_df_clean['transactionDateTime_min']
del tagged_df_clean['transactionDateTime_max']

Encode the transformations into custom transformers for use in a pipeline as follows:

In [ ]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
class NumericCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        self = self
    def fit(self, X, y=None):
        print("NumericCleaner.fit called")
        return self
    def transform(self, X):
        print("NumericCleaner.transform called")
        X["localHour"] = X["localHour"].fillna(-99)
        X["accountAge"] = X["accountAge"].fillna(-1)
        X["numPaymentRejects1dPerUser"] = X["numPaymentRejects1dPerUser"].fillna(-1)
        X.loc[X.loc[:,"localHour"] == -1, "localHour"] = -99
        return X

class CategoricalCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        self = self
    def fit(self, X, y=None):
        print("CategoricalCleaner.fit called")
        return self
    def transform(self, X):
        print("CategoricalCleaner.transform called")
        X = X.fillna(value={"cardType":"U","cvvVerifyResult": "N"})
        X['isUserRegistered'] = X.apply(lambda row: 1 if row["isUserRegistered"] == "TRUE" else 0, axis=1)
        return X

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

numeric_features=["transactionAmountUSD", "localHour", 
                  "transactionIPaddress", "digitalItemCount", "physicalItemCount", "accountAge",
                  "paymentInstrumentAgeInAccount", "numPaymentRejects1dPerUser"
                 ]

categorical_features=["transactionCurrencyCode", "browserLanguage", "paymentInstrumentType", "cardType", "cvvVerifyResult",
                      "isUserRegistered"
                     ]                           

numeric_transformer = Pipeline(steps=[
    ('cleaner', NumericCleaner())
])
                               
categorical_transformer = Pipeline(steps=[
    ('cleaner', CategoricalCleaner()),
    ('encoder', OrdinalEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

Test the transformation pipeline.

In [ ]:
preprocessed_result = preprocessor.fit_transform(tagged_df_clean)

## Train the model

With all the hard work of preparing the data behind you, you are now ready to train the model. In this case you will train a decision tree based ensemble model `GradientBoostingClassifier`.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

X = preprocessed_result
y = tagged_df_clean['label']
X_train, X_test, y_train, y_test = train_test_split(X, y)

gbct = GradientBoostingClassifier()
gbct.fit(X_train, y_train)

Now use the trained model to make predictions against the test set and evaluate the performance.

In [ ]:
y_test_preds = gbct.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, y_test_preds)

## Test save and load of the model

When batch scoring, you will typically work with a model that has been saved off to a shared location. That way, the jobs that use the model for batch processing can easily retrieve the latest version of the model. A good practice is to version that model in Azure Machine Learning service first by registering it. Then any jobs can retrieve the model from the Azure Machine Learning service registry.

Step through the following cells to create some helper functions to prepare for this.

In [ ]:
import os
import azureml
from azureml.core import Workspace
from azureml.core.model import Model

# sklearn.externals.joblib was deprecated in 0.21
from sklearn import __version__ as sklearnver
from packaging.version import Version
if Version(sklearnver) < Version("0.21.0"):
    from sklearn.externals import joblib
else:
    import joblib

In [ ]:
def saveModelToAML(ws, model, model_folder_path="models", model_name="batch-score"):
    # create the models subfolder if it does not exist in the current working directory
    target_dir = './' + model_folder_path
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
  
    # save the model to disk
    joblib.dump(model, model_folder_path + '/' + model_name + '.pkl')
  
    # notice for the model_path, we supply the name of the model outputs folder without a trailing slash
    # anything present in the model folder path will be uploaded to AML along with the model
    print("Registering and uploading model...")
    registered_model = Model.register(model_path=model_folder_path, 
                                      model_name=model_name, 
                                      workspace=ws)
    return registered_model

In [ ]:
def loadModelFromAML(ws, model_name="batch-score"):
  # download the model folder from AML to the current working directory
  model_file_path = Model.get_model_path(model_name, _workspace=ws)
  print('Loading model from:', model_file_path)
  model = joblib.load(model_file_path)
  return model

Save the model to Azure Machine Learning service.

In [ ]:
#Save the model to the AML Workspace
registeredModel = saveModelToAML(ws, gbct)

Now, try out the loading process by getting the model from Azure Machine Learning service, loading the model and then using the model for scoring.

In [ ]:
# Test loading the model

gbct = loadModelFromAML(ws)
y_test_preds = gbct.predict(X_test)

In [ ]:
y_test_preds

In [ ]:
print("subscription_id = '" + ws.subscription_id + "'",
      "resource_group = '" + ws.resource_group + "'",
      "workspace_name = '" + ws.name + "'",
      "workspace_region = '" + ws.location + "'", sep='\n')

> **Important**: Copy the output of the cell above and paste it to Notepad or similar text editor for later.

## Next

Congratulations, you have completed Exercise 3.

After you have **copied the output of the cell above**, that contains connection information to your Azure ML workspace, please return to the Cosmos DB real-time advanced analytics hands-on lab setup guide and continue on to Exercise 4.